<h1>different loaders for loading data</h1>

In [2]:
from langchain.document_loaders import TextLoader
from dotenv import load_dotenv
load_dotenv()


True

In [3]:
text_loader = TextLoader("conversational_RAG_theory.txt");

data = text_loader.load()

In [4]:
# print(data[0].metadata)
# print(data[0])

In [5]:
from langchain_community.document_loaders import PyPDFLoader


In [6]:
loader = PyPDFLoader("2024_Siddhesh_7447664132.pdf")
pages = []
async for page in loader.alazy_load():
    pages.append(page)


resume_data = pages

In [7]:
# print(pages[0])
# print(pages[1])
# print(f"{pages[0].metadata}\n")
# print(pages[0].page_content)

In [8]:
from langchain_community.document_loaders import UnstructuredURLLoader


In [ ]:
# urls = [
#     "https://www.moneycontrol.com/news/telecom/vodafone-idea-bets-big-on-gaming-content-to-widen-4g-subscriber-base-12832202.html",
#     "https://www.moneycontrol.com/news/business/moneycontrol-research/craftsman-automation-significant-headroom-to-grow-valuation-at-fair-level-12832073.html",
# ]
# loader = UnstructuredURLLoader(urls=urls)

# url_data = loader.load()

# len(data)

In [23]:
# url_data[0].metadata

In [24]:
# url_data[1].metadata

<h1>using different techniques for text splitter</h1>

In [12]:
# from langchain.text_splitter import CharacterTextSplitter

# splitter = CharacterTextSplitter(separator = "\n", chunk_size = 200, chunk_overlap = 0)
# chunks = splitter.split_text(data[0].page_content)

In [13]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# r_splitter = RecursiveCharacterTextSplitter(separators = ["\n\n","\n"," "], chunk_size = 200, chunk_overlap = 0)

# docs = chunks = r_splitter.split_text(data[0].page_content)
# len(docs)


In [14]:
# for chunk in chunks:
#     print(str(len(chunk)))


In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

# As data is of type documents we can directly use split_documents over split_text in order to get the chunks.
# docs = text_splitter.split_documents(url_data)
docs = text_splitter.split_documents(resume_data)

In [26]:
len(docs)

3

In [27]:
docs[0]

Document(metadata={'source': '2024_Siddhesh_7447664132.pdf', 'page': 0}, page_content='Siddhesh Sonawane ♂phone+917447664132\nComputer Engineering Student /envel⌢pesiddheshsonawane07@gmail.com\nDr. D.Y. Patil Institute of Technology /githubsiddheshsonawane07\nPune, Maharashtra /userPortfolio\nEducation\n•Bachelor of Computer Engineering 2020-2024\nDr. D.Y. Patil Institute of Technology, Pimpri CGPA: 8.76\nExperience\n•Software Development Intern Remote\nUnicohub Private Limited Oct 2022 - Feb 2023\n•Software Development Intern Remote\nRudraksha Welfare Foundation Feb 2022 - May 2022\nProjects\n•Proctorise\nReact.js, Tensorflow.js, Firebase\n–Developed an online proctoring system utilizing TensorFlow to maintain test integrity.\n–Implemented algorithm for multiple people detection to ensure only authorized test-takers are present.\n–Integrated attentiveness monitoring withTensorFlow Movenet to track user eye movements.\n–Employed facial verification using facial recognition technology t

<h1>embedding the data collected from url into vector data</h1>

In [28]:
from langchain_mistralai import ChatMistralAI

model = ChatMistralAI(model="mistral-large-latest")

In [30]:
from langchain.vectorstores import FAISS
from langchain_mistralai.embeddings import MistralAIEmbeddings

embeddings = MistralAIEmbeddings(model="mistral-embed")

# creating vector store
vectorindex_mistral = FAISS.from_documents(docs, embeddings)


In [31]:
retriever = vectorindex_mistral.as_retriever()

<h1>actual code for basic chatbot on given context</h1>

In [32]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

# Create a retrieval chain to answer questions
document_chain = create_stuff_documents_chain(model, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)


In [33]:
response = retrieval_chain.invoke({"input": "give basic info about the person in the document"})
print(response["answer"])

Based on the provided context, here's the basic information about the person:

**Name:** Siddhesh Sonawane

**Contact Information:**
- Phone: +917447664132
- Email: siddheshsonawane07@gmail.com

**Education:**
- Bachelor of Computer Engineering (2020-2024)
- Institution: Dr. D.Y. Patil Institute of Technology, Pimpri
- CGPA: 8.76

**Location:** Pune, Maharashtra

**Occupation:** Computer Engineering Student

**Experience:**
- Software Development Intern at Unicohub Private Limited (Remote, Oct 2022 - Feb 2023)
- Software Development Intern at Rudraksha Welfare Foundation (Remote, Feb 2022 - May 2022)

**Technical Skills:**
- Programming Languages: Java, JavaScript, Python, Dart
- Backend Skills: Node.js, Express.js, Firebase, MySQL, MongoDB
- Frontend Skills: React, Flutter
- Tools: Git, GitHub, GCP, Azure, Netlify


In [36]:
response = retrieval_chain.invoke({"input": "use basic info about the user and generate FANG resume based latex  "})
print(response["answer"])

Based on the provided context, here's a simplified FANG (Facebook, Amazon, Netflix, Google) resume for Siddhesh Sonawane using LaTeX:

```latex
\documentclass[a4paper,10pt]{article}
\usepackage{latexsym}
\usepackage{xcolor}
\usepackage{hyperref}

\begin{document}

\noindent\textbf{\Large Siddhesh Sonawane}\\
\noindent\href{tel:+917447664132}{+91 7447664132} $\vert$ \href{mailto:siddheshsonawane07@gmail.com}{siddheshsonawane07@gmail.com} $\vert$ \href{https://github.com/siddheshsonawane07}{github.com/siddheshsonawane07} $\vert$ \href{https://userportfolio}{Portfolio}\\
\noindent Pune, Maharashtra

\noindent\rule{\textwidth}{0.4pt}

\noindent\textbf{Education}\\
\noindent\textbf{Dr. D.Y. Patil Institute of Technology, Pimpri}\\
Bachelor of Computer Engineering, \textit{CGPA: 8.76}, 2020-2024

\noindent\rule{\textwidth}{0.4pt}

\noindent\textbf{Experience}\\
\noindent\textbf{Software Development Intern} \hfill \textit{Remote}\\
\noindent Unicohub Private Limited \hfill \textit{Oct 2022 - 

In [39]:
response = retrieval_chain.invoke({"input": "replace only the name with Rushikesh Patil and add dummy school, college, aand masters in the latex"})
print(response["answer"])

```latex
\documentclass{article}
\begin{document}

\section*{Rushikesh Patil}
\section*{Contact Information}
\begin{itemize}
    \item \textbf{Phone:} +917447664132
    \item \textbf{Email:} siddheshsonawane07@gmail.com
    \item \textbf{GitHub:} siddheshsonawane07
    \item \textbf{User Portfolio:} (Not Provided)
\end{itemize}

\section*{Education}
\begin{itemize}
    \item \textbf{Masters in Computer Science} \\
          Dummy University, Dummy Location \\
          2024 - 2026
    \item \textbf{Bachelor of Computer Engineering} \\
          Dr. D.Y. Patil Institute of Technology, Pimpri \\
          2020 - 2024 \\
          CGPA: 8.76
    \item \textbf{High School Diploma} \\
          Dummy High School, Dummy Location \\
          2018 - 2020
\end{itemize}

\section*{Experience}
\begin{itemize}
    \item \textbf{Software Development Intern (Remote)} \\
          Unicohub Private Limited \\
          Oct 2022 - Feb 2023
    \item \textbf{Software Development Intern (Remote)} \\
   

In [43]:
response = retrieval_chain.invoke({"input": "keep the latex structure same, just rename the change from siddhesh sonawane to rushikesh patil"})


In [44]:
print(response["answer"])

```latex
\documentclass{article}
\usepackage{hyperref}

\begin{document}

\section*{Rushikesh Patil}
\subsection*{Contact Information}
\begin{itemize}
    \item Phone: +917447664132
    \item Email: \href{mailto:siddheshsonawane07@gmail.com}{siddheshsonawane07@gmail.com}
    \item GitHub: \href{https://github.com/siddheshsonawane07}{siddheshsonawane07}
    \item Portfolio: \href{https://userPortfolio}{userPortfolio}
\end{itemize}

\subsection*{Education}
\begin{itemize}
    \item \textbf{Bachelor of Computer Engineering} (2020-2024)
    \begin{itemize}
        \item Dr. D.Y. Patil Institute of Technology, Pimpri
        \item CGPA: 8.76
    \end{itemize}
\end{itemize}

\subsection*{Experience}
\begin{itemize}
    \item \textbf{Software Development Intern} (Remote)
    \begin{itemize}
        \item Unicohub Private Limited
        \item Oct 2022 - Feb 2023
    \end{itemize}
    \item \textbf{Software Development Intern} (Remote)
    \begin{itemize}
        \item Rudraksha Welfare Founda